In [ ]:
position = "squareoff"

# Set the initial capital and the current capital
capital = 100000

logging.info("*" * 100)

first_run = True
highest_m2m = 0
has_traded_today = False
has_squareoff_today = False

for i, row in enumerate(df.itertuples()):
    current_datetime_str = str(row.Datetime)[:-6]
    current_datetime_str_short = current_datetime_str
    expiry_datetime = row.closest_expiry
    expiry_datetime = expiry_datetime.replace(hour=15, minute=15, second=0)

    logging.info(f"Current Datetime: {current_datetime_str}")

    # Check if it's a new day and time is 09:30:00+05:30 or it's the first run
    if ('09:45:00' in current_datetime_str or first_run) and not has_traded_today:
        highest_m2m = 0
        atmSP = round_to_nearest_50(row.Close)
        wingPut = atmSP - get_daily_diff(row.Datetime)
        wingCall = atmSP + get_daily_diff(row.Datetime)

        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        
        if first_run:
            df.loc[i, columns_to_reset] = 0
            df.at[i, 'position'] = 'hold'
            position = 'hold'
            df.at[i, 'balance'] = capital
            first_run = False
        else:
            df.at[i, 'position'] = 'beginx'
            position = 'hold'
            calculate_m2m_new(i, start_day=True, caller='beginx 9:45')
            df.at[i, 'stoploss'] = highest_m2m - (100000 * 0.03)
        
        has_traded_today = True  # Mark as traded for the day

    elif df.at[i-1, 'position'] == 'hard-squareoff' and row.Datetime.time() > time(9, 30):
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        df.at[i, 'position'] = position = 'hard-squareoff'
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
        df.at[i, 'totalPL'] = 0

    elif row.Datetime == expiry_datetime:
        capital = df.at[i-1, 'balance']
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'position'] = position = 'squareoff'
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        calculate_m2m_new(i, caller='15:15')
        has_squareoff_today = True  # Mark as squared off for the day

    elif row.Datetime.time() > time(15, 15) and row.Datetime.time() <= time(15, 30):
        df.loc[i, ['atmSP', 'wingCall', 'wingPut']] = [atmSP, wingCall, wingPut]
        df.at[df.index[i], 'position'] = position = 'hold'
        df.at[i, 'balance'] = df.at[i-1, 'balance']

    elif row.Datetime.time() < time(9, 45) and row.Datetime.time() >= time(9, 15):
        df.loc[i, ['atmSP', 'wingCall', 'wingPut']] = [atmSP, wingCall, wingPut]
        df.at[df.index[i], 'position'] = 'hold'
        logging.debug(f"balance at 9:15: {df.at[i-1, 'balance']}")
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        position = 'hold'

    elif df.at[i-1, 'position'] == 'squareoff' and row.Datetime.time() > time(9, 30):
        if row.Datetime.minute % 15 == 0:
            atmSP = round_to_nearest_50(row.Close)
            wingPut = atmSP - get_daily_diff(row.Datetime)
            wingCall = atmSP + get_daily_diff(row.Datetime)
            close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
            df.at[i, 'position'] = position = 'beginx'
            df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
            calculate_m2m_new(i, start_day=True, caller='beginx')
        else:
            df.at[i, 'position'] = position = 'squareoff'
            df.at[i, 'balance'] = df.at[i-1, 'balance']
            df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
            df.at[i, 'totalPL'] = 0

    else:
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        if position == 'hold':
            if row.signal == 'Buy' and row.rsi < 70 and row.ama < row.Close:
                df.at[i, 'position'] = position = 'buy'
                has_traded_today = True
            elif row.signal == 'Sell' and row.rsi > 30 and row.ama > row.Close:
                df.at[i, 'position'] = position = 'sell'
                has_traded_today = True

        elif position == 'buy':
            if row.signal == 'Hold' and row.rsi > 70:
                df.at[i, 'position'] = position = 'squareoff'
                has_traded_today = True
            elif row.signal == 'Sell':
                df.at[i, 'position'] = position = 'squareoff'
                has_traded_today = True

        elif position == 'sell':
            if row.signal == 'Hold' and row.rsi < 30:
                df.at[i, 'position'] = position = 'squareoff'
                has_traded_today = True
            elif row.signal == 'Buy':
                df.at[i, 'position'] = position = 'squareoff'
                has_traded_today = True

        df.at[i, 'position'] = position
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]

        if not has_traded_today:
            df.at[i, 'position'] = df.at[i-1, 'position']

        calculate_m2m_new(i, caller='daily')

        if df.at[i, 'cumReturns'] > highest_m2m:
            highest_m2m = df.at[i, 'cumReturns']

        df.at[i, 'stoploss'] = stoploss_price = highest_m2m - (100000 * 0.02)

        if df.at[i, 'cumReturns'] < stoploss_price:
            logging.info(f" Stoploss hit at {current_datetime_str} with highest m2m: {highest_m2m} and stoploss price: {stoploss_price}")
            logging.info(f" Stoploss hit at {current_datetime_str}")
            df.loc[i, squareoff_columns_reset] = 0
            df.at[i, 'position'] = position = 'hard-squareoff'
            calculate_m2m_new(i, caller='hard-squareoff')
            highest_m2m = df.at[i, 'cumReturns']

save_data()
